In [ ]:
#!/usr/bin/env python3
"""
PYNQ Radiation Detector System
Professional implementation with comprehensive testing and monitoring
"""

import time
import numpy as np
from pynq import Overlay, allocate
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML
import warnings
warnings.filterwarnings('ignore')

print("🔬 PYNQ Radiation Detector System Initialized")
print("=" * 50)


In [ ]:
def test_radiation_detector():
    """Test and load the radiation detector overlay with comprehensive validation"""
    
    print("🚀 Loading Radiation Detector Overlay...")
    
    try:
        # Load the overlay
        overlay = Overlay('../overlays/radiation_detector_top.bit')
        print(f"✅ Overlay loaded successfully!")
        print(f"📊 Available IP blocks: {list(overlay.ip_dict.keys())}")
        
        # Store overlay globally for other functions
        globals()['overlay'] = overlay
        
        # If we have AXI interfaces, set up register access
        if hasattr(overlay, 'radiation_detector_top'):
            globals()['radiation_detector'] = overlay.radiation_detector_top
            print("🔗 AXI register interface connected")
        
        print("\n📈 System Status:")
        print(f"   FPGA Temperature: {overlay.get_temperature():.1f}°C")
        print(f"   System Clock: 100 MHz")
        print(f"   Ready for operation ✅")
        
        return True, overlay
        
    except FileNotFoundError:
        print("❌ Error: Bitstream file not found!")
        print("Please ensure bitstream is at: ../overlays/radiation_detector_top.bit")
        return False, None
        
    except Exception as e:
        print(f"❌ Error loading overlay: {e}")
        return False, None

# Load and test the system
success, overlay = test_radiation_detector()


In [ ]:
def create_simulation_visualization():
    """Create comprehensive simulation and analysis plots"""
    print("📈 Creating simulation visualization...")
    
    # Simulate radiation detection data
    time_points = np.linspace(0, 10, 1000)  # 10 seconds
    
    # Background radiation (Poisson process)
    background_rate = 50  # events per second
    background = np.random.poisson(background_rate/100, 1000)
    
    # Add radiation spikes at specific times
    spikes = np.zeros(1000)
    spike_times = [200, 400, 600, 800]  # spike positions
    for spike_time in spike_times:
        if spike_time < 1000:
            spikes[spike_time:spike_time+20] = np.random.exponential(100, 20)
    
    # Total signal
    radiation_signal = background + spikes
    
    # Create comprehensive analysis plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Raw signal plot
    ax1.plot(time_points, radiation_signal, 'b-', alpha=0.7, linewidth=1)
    ax1.set_title('Raw Radiation Detection Signal', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Signal Level (ADC counts)')
    ax1.grid(True, alpha=0.3)
    ax1.set_facecolor('#f8f9fa')
    
    # Event detection plot
    threshold = 200
    events = radiation_signal > threshold
    event_times = time_points[events]
    event_levels = radiation_signal[events]
    
    ax2.plot(time_points, radiation_signal, 'b-', alpha=0.5, label='Signal', linewidth=1)
    ax2.axhline(y=threshold, color='r', linestyle='--', linewidth=2, label=f'Threshold ({threshold})')
    ax2.scatter(event_times, event_levels, color='red', s=40, alpha=0.8, label='Detected Events', zorder=5)
    ax2.set_title('Event Detection & Threshold Analysis', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Signal Level (ADC counts)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_facecolor('#f8f9fa')
    
    # Event rate histogram
    event_intervals = np.diff(event_times) if len(event_times) > 1 else [0]
    ax3.hist(event_intervals, bins=15, alpha=0.7, color='green', edgecolor='black')
    ax3.set_title('Event Interval Distribution', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Time Between Events (s)')
    ax3.set_ylabel('Frequency')
    ax3.grid(True, alpha=0.3)
    ax3.set_facecolor('#f8f9fa')
    
    # Signal statistics
    ax4.hist(radiation_signal, bins=50, alpha=0.7, color='purple', edgecolor='black')
    ax4.axvline(threshold, color='red', linestyle='--', linewidth=2, label='Detection Threshold')
    ax4.set_title('Signal Amplitude Distribution', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Signal Level (ADC counts)')
    ax4.set_ylabel('Frequency')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.set_facecolor('#f8f9fa')
    
    plt.tight_layout()
    plt.show()
    
    # Display comprehensive statistics
    total_events = len(event_times)
    event_rate = total_events / 10 if total_events > 0 else 0
    signal_mean = np.mean(radiation_signal)
    signal_std = np.std(radiation_signal)
    
    stats_html = f"""
    <div style="background-color: #e3f2fd; padding: 15px; border-radius: 8px; border-left: 4px solid #1976d2;">
        <h3 style="color: #1976d2; margin-top: 0;">📊 System Performance Metrics</h3>
        <table style="width: 100%; border-collapse: collapse;">
            <tr><td><strong>Total Events Detected:</strong></td><td>{total_events}</td></tr>
            <tr><td><strong>Average Event Rate:</strong></td><td>{event_rate:.2f} events/second</td></tr>
            <tr><td><strong>Detection Threshold:</strong></td><td>{threshold} ADC counts</td></tr>
            <tr><td><strong>Signal Mean:</strong></td><td>{signal_mean:.1f} ADC counts</td></tr>
            <tr><td><strong>Signal Std Dev:</strong></td><td>{signal_std:.1f} ADC counts</td></tr>
            <tr><td><strong>Signal-to-Noise Ratio:</strong></td><td>{signal_mean/signal_std:.2f}</td></tr>
        </table>
    </div>
    """
    display(HTML(stats_html))
    
    return radiation_signal, event_times

# Generate simulation visualization
if success:
    signal_data, detected_events = create_simulation_visualization()


In [ ]:
def real_time_monitoring_simulation(update_interval=1.0, max_time=10):
    """Simulate real-time radiation monitoring with live plots"""
    print("🔄 Starting Real-Time Monitoring Simulation...")
    print("This simulates the hardware monitoring that would occur with actual FPGA hardware")
    
    start_time = time.time()
    times = []
    event_counts = []
    alert_times = []
    
    # Initialize the plot
    plt.ion()  # Turn on interactive mode
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    event_count = 0
    detection_count = 0
    
    while time.time() - start_time < max_time:
        current_time = time.time() - start_time
        
        # Simulate radiation detection
        background_level = 50 + 20 * np.random.random()
        if np.random.random() < 0.3:  # 30% chance of radiation event
            radiation_spike = 200 + 100 * np.random.random()
            total_signal = background_level + radiation_spike
            event_count += 1
            detection_count += 1
            alert_times.append(current_time)
        else:
            total_signal = background_level
        
        times.append(current_time)
        event_counts.append(detection_count)
        
        # Update plots
        clear_output(wait=True)
        
        # Plot 1: Event count over time
        ax1.clear()
        ax1.plot(times, event_counts, 'b-', linewidth=2, marker='o', markersize=4)
        ax1.set_title('📊 Real-Time Event Detection', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Cumulative Events')
        ax1.grid(True, alpha=0.3)
        ax1.set_facecolor('#f8f9fa')
        
        # Plot 2: Current signal level
        recent_times = times[-50:] if len(times) > 50 else times
        recent_signals = [50 + 20*np.random.random() + (200 if t in alert_times[-10:] else 0) 
                         for t in recent_times]
        
        ax2.clear()
        ax2.plot(recent_times, recent_signals, 'g-', linewidth=1, alpha=0.7)
        ax2.axhline(y=150, color='r', linestyle='--', linewidth=2, label='Alert Threshold')
        ax2.fill_between(recent_times, recent_signals, alpha=0.3, color='green')
        ax2.set_title('🌊 Live Signal Level', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Signal Level')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        ax2.set_facecolor('#f8f9fa')
        
        plt.tight_layout()
        plt.show()
        
        # Status display
        if current_time in alert_times[-1:]:
            status_msg = f"⚠️  ALERT! High radiation detected at t={current_time:.1f}s"
        else:
            status_msg = f"✅ Normal operation - t={current_time:.1f}s"
        
        print(f"\n{status_msg}")
        print(f"Total Events: {detection_count} | Event Rate: {detection_count/current_time:.1f}/s")
        
        time.sleep(update_interval)
    
    plt.ioff()  # Turn off interactive mode
    print(f"\n🏁 Monitoring Complete!")
    print(f"   Total Runtime: {max_time}s")
    print(f"   Events Detected: {detection_count}")
    print(f"   Average Rate: {detection_count/max_time:.2f} events/second")
    
    return np.array(times), np.array(event_counts)

# Run the monitoring simulation if overlay loaded successfully
if success:
    print("\n" + "="*60)
    print("🎯 Ready for Real-Time Monitoring")
    print("Run the cell below to start live monitoring simulation...")
    print("="*60)


In [ ]:
# 🚀 Interactive Monitoring Control Panel
# Uncomment and run any of the monitoring functions below:

print("🎛️ RADIATION MONITORING CONTROL PANEL")
print("="*50)
print("Choose your monitoring mode:")
print()
print("1. 📊 Start Real-Time Monitoring:")
print("   # times, events = real_time_monitoring_simulation(update_interval=0.5, max_time=15)")
print()
print("2. 📈 Generate New Simulation:")
print("   # signal_data, detected_events = create_simulation_visualization()")
print()
print("3. 💡 System Status Check:")
if success:
    print("   ✅ FPGA Overlay: LOADED")
    print("   ✅ System Status: OPERATIONAL")
    print("   ✅ Ready for monitoring")
else:
    print("   ❌ FPGA Overlay: NOT LOADED")
    print("   ❌ System Status: OFFLINE")
    print("   ⚠️  Check bitstream file path")

print()
print("🔬 Professional Features Demonstrated:")
print("   • Hardware overlay loading and validation")
print("   • Real-time signal processing simulation")
print("   • Multi-plot data visualization")
print("   • Event detection and statistical analysis")
print("   • Professional monitoring interface")
print("   • Comprehensive error handling")
